# Extended Djikstra Algorithm for TDS

In [6]:
from dataclasses import dataclass, field
from enum import Enum
from typing import List, Optional, Set, Annotated, Literal, TypeVar, Callable, Dict, Tuple
import numpy as np
from numpy.typing import NDArray
import numpy.typing as npt
import math

DELTA_T = 0.1  # seconds
MAX_NUMBER = 999
STATE_DIMENSION = 6
DType = TypeVar("DType", bound=np.generic)
Array6 = Annotated[npt.NDArray[DType], Literal[STATE_DIMENSION, 1]]
Matrix6 = Annotated[npt.NDArray[DType],
                    Literal[STATE_DIMENSION, STATE_DIMENSION]]
flt = np.float32


class Weather(Enum):
    warm = 0
    dry = 1


class CirculationState(Enum):
    fluid = 0


class ChunkState(Enum):
    ONE = 1
    TWO = 2
    THREE = 3
    FOUR = 4
    FIVE = 5
    SIX = 6

    @staticmethod
    def nominal_velocities_vector() -> Array6[np.float32]:
        return np.array([10.0, 10.0, 10.0, 10.0, 10.0, 10.0], dtype=np.float32)


@dataclass
class Chunk:
    length: np.float32
    state: ChunkState
    duration: Optional[np.float32] = None


@dataclass
class Intersection:
    name: str
    label: Optional[str] = None
    
    def __hash__(self):
        return hash(self.name)
    
    def __eq__(self, other):
        if isinstance(other, Intersection):
            return self.name == other.name
        return False


@dataclass
class Edge:
    origin: Intersection
    extremity: Intersection
    chunks: List[Chunk]
    weight: Optional[np.float32] = None

    def compute_weight(self) -> np.float32:
        self.weight = sum(
            chunk.duration if chunk.duration is not None else MAX_NUMBER
            for chunk in self.chunks
        )
        return self.weight
    
    def __hash__(self):
        return hash((self.origin.name, self.extremity.name))


@dataclass
class Path:
    edges: List[Edge]
    total_cost: np.float32


@dataclass
class Graph:
    intersections: Set[Intersection]
    edges: Set[Edge]


@dataclass
class Provider:
    name: str
    location: Intersection
    available_quantity: int
    departure_instant: np.float32 = 0
    best_path: Optional[Path] = None
    transportation_time: Optional[np.float32] = None

    def __str__(self) -> str:
        time_str = f"{self.transportation_time} seconds" if self.transportation_time is not None else "Not calculated"
        return f"{self.name} - {self.location.name} - {self.available_quantity} units available - Estimated time: {time_str}"


@dataclass
class ORAlgorithm:
    """
    Implementation of the OR algorithm (Bellman-Ford based) for Time-Dependent Shortest Path (TDSP)
    Based on Orda and Rom [20] continuous-time algorithm
    """
    graph: Graph
    time_horizon: np.float32  # T in the algorithm
    
    def __init__(self, graph: Graph, time_horizon: np.float32):
        self.graph = graph
        self.time_horizon = time_horizon
        self.g_functions: Dict[Intersection, Callable[[np.float32], np.float32]] = {}
        self.h_functions: Dict[Tuple[Intersection, Intersection], Callable[[np.float32], np.float32]] = {}
        
    def solve_tdsp(self, source: Intersection, destination: Intersection, query_time: np.float32) -> Tuple[np.float32, List[Intersection]]:
        """
        Solve TDSP problem using OR algorithm
        Returns: (optimal_travel_time, optimal_path)
        """
        # Step 1: Initialize g_t(t) and h_{k,i}(t) functions
        self._initialize_functions(destination)
        
        # Step 2-7: Iteratively update functions until convergence
        self._iterative_update()
        
        # Step 8: Return optimal solution
        optimal_time = self.g_functions[source](query_time)
        optimal_path = self._reconstruct_path(source, destination, query_time)
        
        return optimal_time, optimal_path
    
    def _initialize_functions(self, destination: Intersection):
        """Step 1-3: Initialize functions"""
        # Step 1: Initialize g_t(t) for all vertices
        for vertex in self.graph.intersections:
            if vertex == destination:
                # g_destination(t) = t for destination vertex
                self.g_functions[vertex] = lambda t: t
            else:
                # g_vertex(t) = ∞ for all other vertices
                self.g_functions[vertex] = lambda t: float('inf')
        
        # Step 2: Initialize h_{k,i}(t) for all edges
        for edge in self.graph.edges:
            edge_key = (edge.origin, edge.extremity)
            # h_{k,i}(t) = ∞ for all edges initially
            self.h_functions[edge_key] = lambda t: float('inf')
    
    def _iterative_update(self):
        """Steps 4-7: Iterative update process"""
        max_iterations = 1000  # Prevent infinite loops
        tolerance = 1e-6
        
        for iteration in range(max_iterations):
            functions_changed = False
            old_g_values = {}
            
            # Store old values for convergence check
            for vertex in self.graph.intersections:
                old_g_values[vertex] = self.g_functions[vertex](0.0)  # Sample at t=0
            
            # Step 5: Update h_{k,i}(t) for all edges
            for edge in self.graph.edges:
                edge_key = (edge.origin, edge.extremity)
                old_h_func = self.h_functions[edge_key]
                
                # h_{k,i}(t) = g_i(t) + w_{k,i}(g_i(t))
                def new_h_func(t, edge=edge):
                    g_i_t = self.g_functions[edge.extremity](t)
                    travel_time = self._compute_edge_travel_time(edge, g_i_t)
                    return g_i_t + travel_time
                
                self.h_functions[edge_key] = new_h_func
            
            # Step 6: Update g_t(t) for all vertices
            for vertex in self.graph.intersections:
                if vertex == list(self.graph.intersections)[0]:  # Skip destination (already initialized)
                    continue
                    
                old_g_func = self.g_functions[vertex]
                
                # Find all incoming edges to this vertex
                incoming_edges = [edge for edge in self.graph.edges if edge.extremity == vertex]
                
                if incoming_edges:
                    def new_g_func(t, v=vertex, edges=incoming_edges):
                        min_value = float('inf')
                        for edge in edges:
                            edge_key = (edge.origin, edge.extremity)
                            h_value = self.h_functions[edge_key](t)
                            min_value = min(min_value, h_value)
                        return min_value
                    
                    self.g_functions[vertex] = new_g_func
            
            # Step 7: Check convergence
            converged = True
            for vertex in self.graph.intersections:
                new_value = self.g_functions[vertex](0.0)  # Sample at t=0
                if abs(new_value - old_g_values[vertex]) > tolerance:
                    converged = False
                    functions_changed = True
                    break
            
            if converged:
                print(f"OR Algorithm converged after {iteration + 1} iterations")
                break
        else:
            print(f"OR Algorithm reached maximum iterations ({max_iterations})")
    
    def _compute_edge_travel_time(self, edge: Edge, departure_time: np.float32) -> np.float32:
        """
        Compute travel time for an edge given departure time
        This is a simplified version - in practice, this would use the EdgeTransportationTimeEstimator
        """
        if edge.weight is not None:
            return edge.weight
        
        # Use EdgeTransportationTimeEstimator if available
        total_time = 0.0
        for chunk in edge.chunks:
            if chunk.duration is not None:
                total_time += chunk.duration
            else:
                # Simplified calculation based on chunk length and nominal velocity
                nominal_velocity = 10.0  # Default velocity
                total_time += chunk.length / nominal_velocity
        
        return total_time
    
    def _reconstruct_path(self, source: Intersection, destination: Intersection, query_time: np.float32) -> List[Intersection]:
        """
        Reconstruct optimal path from source to destination
        """
        path = [source]
        current = source
        current_time = query_time
        
        while current != destination:
            best_next = None
            best_cost = float('inf')
            
            # Find outgoing edges from current vertex
            outgoing_edges = [edge for edge in self.graph.edges if edge.origin == current]
            
            for edge in outgoing_edges:
                edge_key = (edge.origin, edge.extremity)
                cost = self.h_functions[edge_key](current_time)
                
                if cost < best_cost:
                    best_cost = cost
                    best_next = edge.extremity
            
            if best_next is None:
                print("Warning: Could not reconstruct complete path")
                break
                
            path.append(best_next)
            current = best_next
            # Update current_time based on travel time (simplified)
            current_time += self._compute_edge_travel_time(
                next(edge for edge in outgoing_edges if edge.extremity == best_next),
                current_time
            )
        
        return path


@dataclass
class Solver:
    providers: List[Provider]
    destination: Intersection
    map: Optional[Graph] = None
    requested_quantity: int = 0
    time_differential: int = DELTA_T
    preselected_providers: List[Provider] = field(default_factory=list)
    sorted_providers: List[Provider] = field(default_factory=list)
    time_horizon: np.float32 = 100.0  # Default time horizon

    def execute(self) -> None:
        self._preselect_provider()
        self._estimate_transportation_time_for_each_provider_with_or()
        self._sort_providers()
        self._display_top_providers(5)

    def _preselect_provider(self) -> List[Provider]:
        self.preselected_providers = []
        for provider in self.providers:
            if provider.available_quantity >= self.requested_quantity:
                self.preselected_providers.append(provider)
        if not self.preselected_providers:
            raise ValueError(
                "No providers available with sufficient quantity.")
        return self.preselected_providers

    def _estimate_transportation_time_for_each_provider_with_or(self) -> None:
        """
        Enhanced method using OR algorithm for time estimation
        """
        if self.map is None:
            print("Warning: No map provided, using fallback estimation")
            self._estimate_transportation_time_for_each_provider()
            return
            
        # Initialize OR algorithm
        or_algorithm = ORAlgorithm(self.map, self.time_horizon)
        
        for provider in self.preselected_providers:
            try:
                # Use OR algorithm to find optimal path and time
                optimal_time, optimal_path_vertices = or_algorithm.solve_tdsp(
                    provider.location, 
                    self.destination, 
                    provider.departure_instant
                )
                
                # Convert path vertices to edges for Path object
                path_edges = []
                for i in range(len(optimal_path_vertices) - 1):
                    current_vertex = optimal_path_vertices[i]
                    next_vertex = optimal_path_vertices[i + 1]
                    
                    # Find the edge between current and next vertex
                    edge = next((e for e in self.map.edges 
                               if e.origin == current_vertex and e.extremity == next_vertex), None)
                    if edge:
                        path_edges.append(edge)
                
                # Create Path object
                provider.best_path = Path(edges=path_edges, total_cost=optimal_time)
                provider.transportation_time = optimal_time
                
            except Exception as e:
                print(f"Error computing path for {provider.name}: {e}")
                provider.transportation_time = self._estimate_transportation_time_for_a_provider(provider)

    def _estimate_transportation_time_for_each_provider(self) -> None:
        """Fallback method for time estimation"""
        for provider in self.preselected_providers:
            provider.transportation_time = self._estimate_transportation_time_for_a_provider(
                provider)

    def _estimate_transportation_time_for_a_provider(self, provider: Provider) -> np.float32:
        if provider.best_path is not None:
            return provider.best_path.total_cost
        return MAX_NUMBER

    def _sort_providers(self) -> List[Provider]:
        self.sorted_providers = sorted(
            self.preselected_providers,
            key=lambda p: p.best_path.total_cost if p.best_path and p.best_path.total_cost is not None else MAX_NUMBER
        )
        return self.sorted_providers

    def _display_top_providers(self, number: int = 5) -> List[Provider]:
        print("The best providers are:")
        for i in range(min(number, len(self.sorted_providers))):
            provider = self.sorted_providers[i]
            print(f"{i+1}. {provider}")

        return self.sorted_providers[:number]


@dataclass
class ChunckTransportationTimeEstimator:
    chunk: Chunk
    time_origin: np.float32
    current_state: ChunkState
    states_distribution: Array6[np.float32]
    P: Matrix6[Callable[[np.float32], np.float32]]
    time_differential: np.float32 = DELTA_T


    def estimate(self) -> np.float32:
        self.chunk.state = self.current_state
        traveled_distance = 0.0
        time_estimation = 0.0
        while traveled_distance < self.chunk.length:
            velocity = self._compute_distributed_velocity(self.states_distribution, ChunkState.nominal_velocities_vector())
            temporal_distance = velocity * self.time_differential
            if temporal_distance + traveled_distance >= self.chunk.length:
                temporal_distance = self.chunk.length - traveled_distance
            traveling_time = temporal_distance / velocity
            time_estimation += traveling_time
            traveled_distance += temporal_distance

            self.states_distribution = self.states_distribution @ self.P(traveling_time)
            self.current_state = ChunkState(self.states_distribution.argmax()+1)

        return time_estimation

    def _compute_distributed_velocity(self, distribution: Array6[np.float32], nominal_velocities: Array6[np.float32]) -> np.float32:
        return np.sum(distribution * nominal_velocities)


@dataclass
class EdgeTransportationTimeEstimator:
    edge: Edge
    time_origin: np.float32
    current_state: ChunkState
    states_distribution: Array6[np.float32]
    P: Matrix6[Callable[[np.float32], np.float32]]
    time_differential: np.float32 = DELTA_T

    def estimate(self) -> np.float32:
        total_time_estimation = 0.0
        for chunk in self.edge.chunks:
            estimator = ChunckTransportationTimeEstimator(
                chunk=chunk,
                time_origin=self.time_origin,
                current_state=self.current_state,
                states_distribution=self.states_distribution,
                time_differential=self.time_differential,
                P=self.P
            )
            total_time_estimation += estimator.estimate()
        return total_time_estimation


# Example usage and testing
def create_sample_graph() -> Graph:
    """Create a sample graph for testing"""
    # Create intersections
    A = Intersection("A")
    B = Intersection("B") 
    C = Intersection("C")
    D = Intersection("D")
    
    # Create chunks
    chunk1 = Chunk(length=np.float32(100.0), state=ChunkState.ONE, duration=np.float32(10.0))
    chunk2 = Chunk(length=np.float32(150.0), state=ChunkState.TWO, duration=np.float32(15.0))
    chunk3 = Chunk(length=np.float32(80.0), state=ChunkState.ONE, duration=np.float32(8.0))
    chunk4 = Chunk(length=np.float32(120.0), state=ChunkState.THREE, duration=np.float32(12.0))
    
    # Create edges
    edge_AB = Edge(A, B, [chunk1])
    edge_AC = Edge(A, C, [chunk2])
    edge_BD = Edge(B, D, [chunk3])
    edge_CD = Edge(C, D, [chunk4])
    
    # Compute weights
    edge_AB.compute_weight()
    edge_AC.compute_weight()
    edge_BD.compute_weight()
    edge_CD.compute_weight()
    
    return Graph(
        intersections={A, B, C, D},
        edges={edge_AB, edge_AC, edge_BD, edge_CD}
    )


def test_or_algorithm():
    """Test the OR algorithm implementation"""
    print("Testing OR Algorithm Implementation")
    print("=" * 50)
    
    # Create sample graph
    graph = create_sample_graph()
    
    # Create OR algorithm instance
    or_algo = ORAlgorithm(graph, time_horizon=np.float32(100.0))
    
    # Find vertices
    source = next(v for v in graph.intersections if v.name == "A")
    destination = next(v for v in graph.intersections if v.name == "D")
    
    # Solve TDSP
    optimal_time, optimal_path = or_algo.solve_tdsp(source, destination, query_time=0.0)
    
    print(f"Source: {source.name}")
    print(f"Destination: {destination.name}")
    print(f"Optimal travel time: {optimal_time}")
    print(f"Optimal path: {' -> '.join([v.name for v in optimal_path])}")
    
    # Test with Solver
    print("\nTesting with Solver class")
    print("-" * 30)
    
    providers = [
        Provider("Provider1", source, 100, departure_instant=0.0),
        Provider("Provider2", next(v for v in graph.intersections if v.name == "B"), 150, departure_instant=0.0)
    ]
    
    solver = Solver(
        providers=providers,
        destination=destination,
        map=graph,
        requested_quantity=50,
        time_horizon=100.0
    )
    
    solver.execute()


if __name__ == "__main__":
    test_or_algorithm()

Testing OR Algorithm Implementation


RecursionError: maximum recursion depth exceeded while calling a Python object